# Import libraries

In [148]:
%config Completer.use_jedi = False
import glob
import os
import cv2
from natsort import natsorted
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

Tensorflow version: 2.4.1
Keras version: 2.4.0


# Get path to images and masks into a list

In [88]:
def load_data(img_type, dir_path):
    """ Glob filenames and add them to a list.
    Provide img_type == 'image' (*.jpg) or 'mask' (*.png). 
    *_x contains images and *_y contains masks """
    
    suffix = ''
    
    if img_type == 'image':
        suffix = '.jpg'
    else:
        suffix = '.png'
    
    
    list_files = natsorted(glob.glob(dir_path+'/*'+suffix))
    
    return list_files

In [153]:
dataset_path = 'partial_dataset/'

train_masks = os.path.join(dataset_path, 'train', 'labels')
train_images = os.path.join(dataset_path, 'train', 'images')
train_x, train_y = load_data('image',
                             train_images), load_data('mask', train_masks)

val_masks = os.path.join(dataset_path, 'val', 'labels')
val_images = os.path.join(dataset_path, 'val', 'images')
val_x, val_y = load_data('image', val_images), load_data('mask', val_masks)

print(
    f"Train images: {len(train_x)}, Train masks: {len(train_y)}, Val images: {len(val_x)}, Val masks: {len(val_y)}"
)

Train images: 12901, Train masks: 12901, Val images: 4431, Val masks: 4431


In [149]:
from sklearn.model_selection import train_test_split

In [154]:
train_x, val_x = train_test_split(val_x, test_size=0.3, random_state=88)
train_y, val_y = train_test_split(val_y, test_size=0.3, random_state=88)
print(
    f"Train images: {len(train_x)}, Train masks: {len(train_y)}, Val images: {len(val_x)}, Val masks: {len(val_y)}"
)

Train images: 3101, Train masks: 3101, Val images: 1330, Val masks: 1330


# Make a `tf` dataset

In [155]:
H, W = 256, 256

# ------------------------------------------------------------------------------
def read_image(data_x):
    """Read path of image.
    data_x ==  *_x
    """

    x = cv2.imread(data_x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    
    return x


# ------------------------------------------------------------------------------
def read_mask(data_y):
    """Read path of mask.
    data_y == *_y
    """
    y = cv2.imread(data_y, cv2.IMREAD_GRAYSCALE)
    y = cv2.resize(y, (H, W))
    y = y - 1
    y = y.astype(np.int32)
    
    return y


# ------------------------------------------------------------------------------
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        image = read_image(x)
        mask = read_mask(y)

        return image, mask

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    mask = tf.one_hot(mask, 3, dtype=tf.int32)
    image.set_shape([H, W, 3])
    mask.set_shape([H, W, 3])

    return image, mask


# ------------------------------------------------------------------------------
def make_tf_dataset(x, y, batch=8):
    """Make tensorflow dataset.
    """
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2)

    return dataset


# ------------------------------------------------------------------------------

In [156]:
batch_size=8
train_dataset = make_tf_dataset(train_x, train_y, batch=batch_size)
val_dataset = make_tf_dataset(val_x, val_y, batch=batch_size)

In [157]:
train_dataset

<PrefetchDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.int32)>

In [158]:
val_dataset

<PrefetchDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.int32)>

# Build a U-NET Image Segmentation Model

In [159]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import Model

In [160]:
# ------------------------------------------------------------------------------
def conv_block(inputs, filters, pool=True):
    x = Conv2D(filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    if pool == True:
        p = MaxPool2D((2, 2))(x)
        return x, p
    else:
        return x


# ------------------------------------------------------------------------------
def build_unet(shape, num_classes):
    inputs = Input(shape)

    """Encoder"""
    x1, p1 = conv_block(inputs, 16, pool=True)
    x2, p2 = conv_block(p1, 32, pool=True)
    x3, p3 = conv_block(p2, 48, pool=True)
    x4, p4 = conv_block(p3, 64, pool=True)

    """Bridge"""
    b1 = conv_block(p4, 128, pool=False)

    """Decoder"""
    u1 = UpSampling2D((2,2), interpolation="bilinear")(b1)
    # print(u1.shape, x4.shape)
    c1 = Concatenate()([u1, x4])
    x5 = conv_block(c1, 64, pool=False)

    u2 = UpSampling2D((2,2), interpolation="bilinear")(x5)
    c2 = Concatenate()([u2, x3])
    x6 = conv_block(c2, 48, pool=False)

    u3 = UpSampling2D((2,2), interpolation="bilinear")(x6)
    c3 = Concatenate()([u3, x2])
    x7 = conv_block(c3, 32, pool=False)

    u4 = UpSampling2D((2,2), interpolation="bilinear")(x7)
    c4 = Concatenate()([u4, x1])
    x8 = conv_block(c4, 16, pool=False)

    """Output layer"""
    output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

    return Model(inputs, output)


# ------------------------------------------------------------------------------

In [161]:
model = build_unet((256, 256, 3), 2)
model.summary()

Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_627 (Conv2D)             (None, 256, 256, 16) 448         input_34[0][0]                   
__________________________________________________________________________________________________
batch_normalization_594 (BatchN (None, 256, 256, 16) 64          conv2d_627[0][0]                 
__________________________________________________________________________________________________
activation_594 (Activation)     (None, 256, 256, 16) 0           batch_normalization_594[0][0]    
___________________________________________________________________________________________

# Train the model

In [76]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [162]:
"""Seeding"""
np.random.seed(88)
tf.random.set_seed(88)


"""Hyperparameter"""
shape = (256, 256, 3)
num_classes = 3
lr = 1e-4
batch_size = 8
epochs = 3


"""Model"""
model = build_unet(shape, num_classes)
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr))
train_dataset = make_tf_dataset(train_x, train_y, batch=batch_size)
val_dataset = make_tf_dataset(val_x, val_y, batch=batch_size)

train_steps = len(train_x)//batch_size
val_steps = len(val_x)//batch_size

callbacks = [
             ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
             ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1, verbose=1, min_lr = 1e-6),
             EarlyStopping(monitor='val_loss', patience=5, verbose=1)
]

model.fit(
    train_dataset,
    steps_per_epoch=train_steps,
    validation_data=val_dataset,
    validation_steps=val_steps,
    epochs=epochs,
    callbacks = callbacks
)

Epoch 1/3
387/387 [==============================] - 1105s 3s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.00000, saving model to model.h5
Epoch 2/3
387/387 [==============================] - 973s 3s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.00000
Epoch 3/3
387/387 [==============================] - 969s 3s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00

Epoch 00003: val_loss did not improve from 0.00000
